Wstępny kod z funkcjami, ktorych nie mozemy uzywac, zeby zobaczyc czy wyjdzie

In [27]:
import cv2
import numpy as np

# Ładowanie wszystkich 5 obrazów i usuwanie ludzi
images = []
for i in range(1, 6):
    image_path = f"cathedral/d00{i}.jpg"
    img = cv2.imread(image_path)
    images.append(img)

# Obliczanie współrzędnych punktów, w których obrazy zostaną nakładane na siebie
height, width, _ = images[0].shape
combined_image = np.zeros((height, width, 3), dtype=np.uint8)
kernel = np.ones((4, 4), np.uint8) 


# Znajdowanie najczęstszego piksela dla każdego piksela pomiędzy obrazami
for y in range(height):
    for x in range(width):
        pixel_values = [tuple(img[y, x]) for img in images]
        most_frequent_value = max(set(pixel_values), key=pixel_values.count)
        combined_image[y, x] = most_frequent_value
print('good')
# Calculate histogram of the combined image
histogram = cv2.calcHist([combined_image], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

# Find the peak(s) in the histogram
peaks = np.where(histogram > np.max(histogram) * 0.007) #Mozna pobawic sie jak duze peaki ma wylapywac zmieniajac liczbe ktora mnozymy, dobre sa z zakresu 0.01 - 0.004

# Customized mask based on color similarity 
# testowa maska ktora probuje brac pod uwage podobienstwo kolorow, bardzo powolna i nawet nie zdazylem jej odpalic

mask = np.zeros((height, width), dtype=np.uint8)

# Threshold the image based on the peak values 
#prosta maska ktora tylko usuwa peaki, dziala okej
mask = np.zeros((height, width), dtype=np.uint8)
for i in range(len(peaks[0])):
    mask[np.where((combined_image[:, :, 0] == peaks[0][i]) & (combined_image[:, :, 1] == peaks[1][i]) & (combined_image[:, :, 2] == peaks[2][i]))] = 255

# Apply the mask to remove unwanted pixels
result_image = cv2.bitwise_and(combined_image, combined_image, mask=mask)

# Inpainting to replace removed pixels  #ten inpainting slabo mi dzialal takze trzeba zrobic funkcje do wypelniania
#result_image = cv2.inpaint(result_image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

# Zapisanie obrazu bez ludzi
cv2.imwrite("combined_no_people_overlaid.jpg", cv2.filter2D(cv2.dilate(result_image,kernel), -1,kernel/np.sum(kernel)))


good


True